In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Silver Data Tranformation

In [0]:
df = spark.read.format("delta").option("header",True).option("inferSchema",True).load("abfss://bronze@netflixprojectstrg.dfs.core.windows.net/netflix_titles")

In [0]:
df.display()

In [0]:
df = df.fillna({"duration_minutes": 0, "duration_seasons": 1})

In [0]:
df.display()

In [0]:
df= df.withColumn("duration_minutes", when(col("duration_minutes").try_cast(IntegerType()).isNull(),0).otherwise(col("duration_minutes").cast(IntegerType())))\
      .withColumn("duration_seasons", when(col("duration_seasons").try_cast(IntegerType()).isNull(),0).otherwise(col("duration_seasons").cast(IntegerType())))


In [0]:
df.display()


In [0]:
display(df.filter(col("duration_minutes").isNull()).count())

In [0]:
df = df.withColumn("shorttitle",split(col("title"), ":")[0])

In [0]:
display(df)

In [0]:
df = df.withColumn("rating",split(col("rating"), "-")[0])
df.display()

In [0]:
df = df.withColumn("type_flag", when(col("type") == "Movie", 1)\
                                .when(col("type") == "TV Show", 2)\
                                .otherwise(0))
df.display()

In [0]:
from pyspark.sql.window import Window

In [0]:
df = df.withColumn("duration_ranking",dense_rank().over(Window.orderBy(col("duration_minutes").desc())))

In [0]:
df.display()

In [0]:
df.createOrReplaceTempView("temp_view")

In [0]:
df = spark.sql("""
                SELECT * FROM temp_view
               """)
df.display()

In [0]:
df_vis = df.groupBy("type").agg(count("*").alias("total_count"))
df_vis.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df.write.format("delta").mode("overwrite").option("path","abfss://silver@netflixprojectstrg.dfs.core.windows.net/netflix_titles").save()
